In [1]:
%load_ext autoreload


In [2]:
%autoreload 2

In [18]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation

In [155]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2014-08-21/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
files_start = 1
files_end = 10

In [80]:
# cup configuration
#cup_config = cycle_Sb
cup_config = cycles1
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "126"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycles1_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"], ["117", "118", "119", "122", "124"]]
isotopes = [["112", "114", "115", "116", "117", "118", "119", "122", "124"]]
#isotopes = [["116", "117", "118", "119", "122", "124"]]
denom_isotope = "120"
data_sample_column = [(i + "/" + "120") for i in isotopes[0]]
print data_sample_column
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False

['112/120', '114/120', '115/120', '116/120', '117/120', '118/120', '119/120', '122/120', '124/120']


In [36]:
columns = ["cycle", "sample", "date", "H9 (1)","H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]
df_all = pd.DataFrame()
files_1 = range(files_start, files_end, 1)
for sample in files_1:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero = pd.DataFrame(df_zero["cycle1"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero["sample"] = sample_name
    df_zero["cycle"] = df_zero.index.values
    df_zero.index = df_zero["date"]
    df_zero = df_zero[columns]
    df_all = df_all.append(df_zero, ignore_index = True)

df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),Ax (1),L1 (1),L2 (1),L3 (1),L4 (1)
date,,,,,,,,,,,,,,,,
2014-08-21 16:04:00,1,Sn Specpure 100ppb,0.000169,1.700762,1.320661,9.014462,2.337953,6.492650,2.023877,3.776931,0.086604,0.166584,0.000382,0.237559,0.000410,0.000394
2014-08-21 16:04:00,2,Sn Specpure 100ppb,0.000143,1.688658,1.311239,8.950914,2.321462,6.447094,2.009689,3.750717,0.086033,0.165359,0.000385,0.235897,0.000430,0.000412
2014-08-21 16:04:00,3,Sn Specpure 100ppb,0.000151,1.683193,1.307000,8.922417,2.314152,6.426521,2.003303,3.738650,0.085738,0.164940,0.000379,0.235197,0.000412,0.000422
2014-08-21 16:04:00,4,Sn Specpure 100ppb,0.000158,1.685737,1.309050,8.934955,2.317410,6.435421,2.006086,3.743840,0.085854,0.165273,0.000421,0.235790,0.000408,0.000440
2014-08-21 16:04:00,5,Sn Specpure 100ppb,0.000137,1.680045,1.305290,8.909241,2.310581,6.416700,2.000207,3.730672,0.085559,0.164617,0.000371,0.234700,0.000391,0.000433
2014-08-21 16:04:00,6,Sn Specpure 100ppb,0.000156,1.643132,1.275835,8.709365,2.258738,6.272876,1.955376,3.649221,0.083744,0.160666,0.000384,0.229717,0.000392,0.000381
2014-08-21 16:04:00,7,Sn Specpure 100ppb,0.000173,1.651965,1.282645,8.754163,2.270298,6.304542,1.965098,3.667212,0.084113,0.161745,0.000386,0.230642,0.000380,0.000384
2014-08-21 16:04:00,8,Sn Specpure 100ppb,0.000147,1.686005,1.309022,8.933380,2.316682,6.433127,2.005159,3.741902,0.085814,0.164933,0.000377,0.235252,0.000417,0.000381
2014-08-21 16:04:00,9,Sn Specpure 100ppb,0.000176,1.678784,1.303401,8.896041,2.307152,6.406509,1.996950,3.726249,0.085477,0.164299,0.000372,0.234349,0.000421,0.000420


In [102]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_external_norm = pd.DataFrame()
df_external_norm_mean = pd.DataFrame()

for sample in files_1:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    #blk_1 = NU_data_read(path, files[sample][1][0], cup_config)
    #blk_2 = NU_data_read(path, files[sample][1][1], cup_config)

    df_zero = df.data_zero_corr(sample)
    #df_bgd_1 = blk_1.data_zero_corr(files[sample][1][0])
    #df_bgd_2 = blk_2.data_zero_corr(files[sample][1][1])

    new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_1 , denom_corr_ratio)
    #new_corr.line2_corr(df_zero, "119")
    #new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
    
    data_sample_raw = new_corr.raw_signals(denom_isotope)
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = data_sample_raw[np.abs(data_sample_raw-data_sample_raw.mean())<=(2*data_sample_raw.std())]
    
    data_sample = new_corr.internal_norm_1(denom_isotope, 10)
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = data_sample[np.abs(data_sample-data_sample.mean())<=(2*data_sample.std())]
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample.drop(denom_isotope, axis=1)
    data_sample.columns = data_sample_column
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)
    
    df_external_norm = df_external_norm.append(data_sample)

In [159]:
#df_external_norm.set_index("date", inplace = True)
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_external_sub = pd.DataFrame(df_external_norm, columns = columns)
df_external_sub.to_csv(path_sav + "Sn_H9_L4_internal_corr_all.csv")

In [160]:
df_external_norm_mean = df_external_sub.drop("cycle", axis=1).groupby(df_external_sub.index).agg([np.mean, np.std])
df_external_norm_mean["sample"] = df_external_norm[df_external_norm["cycle"] == 1]["sample"]
df_external_norm_mean.to_csv(path_sav + "Sn_H9_L4_internal_corr_mean.csv")

In [119]:
sub_index = df_external_norm_mean[(df_external_norm_mean.sample.shift(1) == df_external_norm_mean.sample.shift(-1)) & (df_external_norm_mean.sample.shift(1) == "NIST 100ppb")].index
sample = df_external_norm_mean["sample"]
new = df_external_norm_mean[df_external_norm_mean.columns[df_external_norm_mean.dtypes != 'object']]

new =  ((new / ((new.shift(1) + new.shift(-1))/2))-1)*1000
new["sample"] = sample

new = new[new.index.isin(sub_index)]
#df_external_norm_signal = df_external_norm_signal[df_external_norm_signal['sample'].isin(sub_index.tolist())]
#df_new_signal = new.merge(df_external_norm_signal, left_index=True, right_index=True)
#new[new.sample != "NIST 100ppb"]


In [123]:
new.groupby("sample").agg([np.mean, np.std, np.size])

116/120                     117/120                 \
                        mean       std size         mean       std size   
sample                                                                    
BHVO-2 1_1(2) I    -0.562374       NaN    1    -0.435635       NaN    1   
BHVO-2 1_2(1) I    -0.761249       NaN    1    -0.576920       NaN    1   
BHVO-2 2-2 I       -0.630289       NaN    1    -0.514774       NaN    1   
BHVO-2 2_1 I       -0.638599       NaN    1    -0.507914       NaN    1   
J1 1_1 I           -0.479768  0.001827    2    -0.366074  0.020097    2   
J2 1_1 I           -0.337210       NaN    1    -0.287974       NaN    1   
NIST 100ppb         0.005390  0.013163    3    -0.011433  0.036562    3   
SPEX 100ppb        -0.280679  0.015528   45    -0.208626  0.040104   45   
SPEX1 1_1(1) I   3826.688675       NaN    1  2233.664489       NaN    1   
SPEX1 2_1 I        -0.385132       NaN    1    -0.306317       NaN    1   
SPEX1 II 3_1 I     -0.411635       NaN    1    -0.305280       NaN    1   
SPEX1 II 4_1 I     -0.522829       NaN    1    -0.372163       NaN    1   

                     118/120                    119/120                 \
                        mean       std size        mean       std size   
sample                                                                   
BHVO-2 1_1(2) I    -0.293450       NaN    1   -0.142375       NaN    1   
BHVO-2 1_2(1) I    -0.389528       NaN    1   -0.207285       NaN    1   
BHVO-2 2-2 I       -0.213156       NaN    1   -0.087376       NaN    1   
BHVO-2 2_1 I       -0.219054       NaN    1   -0.113395       NaN    1   
J1 1_1 I           -0.219902  0.019291    2   -0.079692  0.016256    2   
J2 1_1 I           -0.178934       NaN    1   -0.028617       NaN    1   
NIST 100ppb         0.025868  0.023632    3   -0.024272  0.032934    3   
SPEX 100ppb        -0.138885  0.016831   45   -0.073155  0.019215   45   
SPEX1 1_1(1) I   1177.724175       NaN    1  482.421221       NaN    1   
SPEX1 2_1 I        -0.206499       NaN    1   -0.082116       NaN    1   
SPEX1 II 3_1 I     -0.205526       NaN    1   -0.098153       NaN    1   
SPEX1 II 4_1 I     -0.246235       NaN    1   -0.120331       NaN    1   

                    122/120                    124/120                 
                       mean       std size        mean       std size  
sample                                                                 
BHVO-2 1_1(2) I    0.234118       NaN    1    0.529079       NaN    1  
BHVO-2 1_2(1) I    0.359455       NaN    1    0.720012       NaN    1  
BHVO-2 2-2 I       0.293040       NaN    1    0.485557       NaN    1  
BHVO-2 2_1 I       0.297594       NaN    1    0.478529       NaN    1  
J1 1_1 I           0.306541  0.080122    2    0.544771  0.074697    2  
J2 1_1 I           0.154632       NaN    1    0.344715       NaN    1  
NIST 100ppb       -0.032520  0.034205    3   -0.028265  0.011089    3  
SPEX 100ppb        0.131179  0.016331   45    0.259360  0.017111   45  
SPEX1 1_1(1) I  -527.178495       NaN    1 -784.334288       NaN    1  
SPEX1 2_1 I        0.167485       NaN    1    0.355228       NaN    1  
SPEX1 II 3_1 I     0.195785       NaN    1    0.367522       NaN    1  
SPEX1 II 4_1 I     0.288305       NaN    1    0.490732       NaN    1